In [1]:
magma_dir = '/home/ubuntu/magma/'
bucket_dir = '/home/ubuntu/s3/'
transformers_dir = '/home/ubuntu/transformers/'
cache_dir = bucket_dir+'.cache/'

## **Fine-tuning**

In [2]:
finetune_script = '"'+transformers_dir+'examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+transformers_dir+'examples/seq2seq/run_eval.py"'

### **Config**

In [3]:
import sys
sys.path.insert(0, magma_dir)
import config

import wandb
wandb.login()

project_name = 'ft_pegasus_para_wordembed'
%env WANDB_PROJECT=$project_name

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: marcoabrate (use `wandb login --relogin` to force relogin)


env: WANDB_PROJECT=ft_pegasus_para_wordembed


### Karger Books Para Wordembed

In [4]:
model_name_or_path = 'google/pegasus-large'

In [5]:
data_dir = '"'+bucket_dir+'datasets/karger_books_para_wordembed/pegasus/st/"'

output_dir = '"'+bucket_dir+'fine-tuning/'+project_name+'"'

log_dir = output_dir + '/logs'

In [6]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.num_beams = 2

model_config_dir = '"'+bucket_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

##### Fine tune

In [7]:
'''
model_name_or_path = output_dir[:-1] + 'checkpoint-325"'
print(model_name_or_path)

from transformers import AutoTokenizer
tok = AutoTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6', use_cache=False)
tok.save_pretrained(model_name_or_path[1:-1])
'''

'\nmodel_name_or_path = output_dir[:-1] + \'checkpoint-325"\'\nprint(model_name_or_path)\n\nfrom transformers import AutoTokenizer\ntok = AutoTokenizer.from_pretrained(\'sshleifer/distilbart-cnn-12-6\', use_cache=False)\ntok.save_pretrained(model_name_or_path[1:-1])\n'

In [8]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--cache_dir $cache_dir \
--data_dir $data_dir \
--freeze_embeds \
--adafactor \
--do_train \
--learning_rate 1e-4 \
--label_smoothing_factor 0.1 \
--warmup_steps 0 \
--fp16 \
--sortish_sampler \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--num_train_epochs 10 \
--logging_steps 20 --logging_first_step \
--per_device_train_batch_size 2 --per_device_eval_batch_size 8 \
--gradient_accumulation_steps 16 \
--evaluation_strategy steps --eval_steps 14 --eval_beams 2 \
--predict_with_generate \
--save_steps 5000 \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

02/12/2021 11:44:10 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
02/12/2021 11:44:10 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/home/ubuntu/s3/fine-tuning/ft_pegasus_para_wordembed', overwrite_output_dir=True, do_train=True, do_eval=None, do_predict=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=8, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=16, eval_accumulation_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_steps=0, logging_dir='runs/Feb12_11-44-10_ip-172-31-39-35', logging_first_step=True, logging_steps=20, save_steps=5000, save_total_limit=None

[INFO|tokenization_utils_base.py:1786] 2021-02-12 11:44:12,714 >> loading file https://huggingface.co/google/pegasus-large/resolve/main/spiece.model from cache at /home/ubuntu/s3/.cache/66f187d645734a6204f3fd24593fbf0d9e36b528dd85b3adae9a566b17b4768f.1acf68c74589da6c7fa3548093824dfc450a54637f4356929bbfea7e294a68f8
[INFO|tokenization_utils_base.py:1786] 2021-02-12 11:44:12,715 >> loading file https://huggingface.co/google/pegasus-large/resolve/main/tokenizer.json from cache at None
[INFO|tokenization_utils_base.py:1786] 2021-02-12 11:44:12,715 >> loading file https://huggingface.co/google/pegasus-large/resolve/main/added_tokens.json from cache at None
[INFO|tokenization_utils_base.py:1786] 2021-02-12 11:44:12,715 >> loading file https://huggingface.co/google/pegasus-large/resolve/main/special_tokens_map.json from cache at /home/ubuntu/s3/.cache/fbf9c7cf2d49b24712b53a2760e7c62a2acecd1496908822df00b8ec2683ca6d.294ebaa4cd17bb284635004c92d2c4d522ec488c828dcce0c2471b6f28e3fe82
[INFO|tokeniza

wandb:                                                                                
wandb: Find user logs for this run at: /home/ubuntu/magma/fine-tuning/wandb/run-20210212_114510-2a0vbrbz/logs/debug.log
wandb: Find internal logs for this run at: /home/ubuntu/magma/fine-tuning/wandb/run-20210212_114510-2a0vbrbz/logs/debug-internal.log
wandb: Run summary:
wandb:            train/loss 3.3785
wandb:   train/learning_rate 0.0001
wandb:           train/epoch 0.03
wandb:                 _step 1
wandb:              _runtime 13
wandb:            _timestamp 1613130323
wandb: Run history:
wandb:            train/loss ▁
wandb:   train/learning_rate ▁
wandb:           train/epoch ▁
wandb:                 _step ▁
wandb:              _runtime ▁
wandb:            _timestamp ▁
wandb: 
wandb: Synced 5 W&B file(s), 1 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: 
wandb: Synced /home/ubuntu/s3/fine-tuning/ft_pegasus_para_wordembed: https://wandb.ai/marcoabrate/ft_pegasus_para_wordembed/